In [ ]:
import hashlib
import math

class LFSR:
    def __init__(self, seed, width):
        self.state = seed & ((1 << width) - 1)
        self.width = width

    def next_bit(self):
        # Simple x^n + x^(n-1) + 1 LFSR feedback (can be replaced with desired polynomial)
        feedback = ((self.state >> (self.width - 1)) ^ (self.state >> (self.width - 2))) & 1
        self.state = ((self.state << 1) | feedback) & ((1 << self.width) - 1)
        return self.state & 1

    def next_bits(self, n):
        return [(self.next_bit()) for _ in range(n)]

def shake_stream(seed, length):
    """Generate pseudo-random bitstream using SHAKE-256"""
    shake = hashlib.shake_256(seed.to_bytes((seed.bit_length()+7)//8, 'big'))
    byte_len = (length + 7) // 8
    stream = shake.digest(byte_len)
    bits = []
    for b in stream:
        for i in range(8):
            bits.append((b >> i) & 1)
    return bits[:length]

def generate_templates(lfsr, r, N):
    """Generate N templates of length r from LFSR"""
    templates = []
    for _ in range(N):
        template = lfsr.next_bits(r)
        templates.append(template)
    return templates

def tms_embed(K, seed_SK, lfsr_width, r, N, carrier_length=1024):
    """
    Embed 16-bit TMS K[0..15] into a SHAKE carrier using session-LFSR templates
    """
    lfsr = LFSR(seed_SK, lfsr_width)
    templates = generate_templates(lfsr, r, N)
    B = []
    i = 0  # TMS bit index
    sel_bits = math.ceil(math.log2(N))

    # Generate carrier stream using SHAKE-256
    carrier = shake_stream(seed_SK, carrier_length)

    while i < 16:
        b = carrier[len(B) % carrier_length]  # next bit from SHAKE stream
        B.append(b)

        if len(B) >= sel_bits:
            # Select template using last sel_bits
            sel_val = 0
            for j in range(sel_bits):
                sel_val |= B[-sel_bits + j] << (sel_bits - j - 1)
            sel = sel_val % N
            t_sel = templates[sel]

            if len(B) >= r and B[-r:] == t_sel:
                B.append(K[i])
                i += 1
    return B

# Example usage:
if __name__ == "__main__":
    K = [1,0,1,0,1,1,0,0,1,0,0,1,1,0,1,0]  # 16-bit TMS
    seed_SK = 0xACE1
    lfsr_width = 16
    r = 4
    N = 2
    encrypted_bitstream = tms_embed(K, seed_SK, lfsr_width, r, N)
    print("Encrypted bitstream:", encrypted_bitstream)
